# EDA of the crawled data

## #1: What does every day's job postings look like?

Pick a date and peek at the CSV. 

In [2]:
import pandas as pd
import os

_data0 = pd.read_csv("../data/crawled_jobs_2023-10-22.csv")
_data0

,job_title,required_skills,job_type_1,job_type_2,linkedin_url,company,company_linkedin_url,location,posted_date,applicant_count,job_description,benefits
0,RWE Scientist / Epidemiologist,"Customer Relationship Management (CRM), Epidem...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3737909849/...,MedEngine,https://www.linkedin.com/company/medengine/life,"MedEngine · Helsinki, Uusimaa, Finland 1 week...",1 week ago,0,About the job\nMedEngine is a digitally minded...,NaN
1,Data Engineer,"Data Engineering, Git, Python (Programming Lan...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3736532279/...,Suomen Palloliitto - Football Association of F...,https://www.linkedin.com/company/football-asso...,Suomen Palloliitto - Football Association of F...,1 week ago,0,About the job\nDATA ENGINEER\n\nSUOMEN PALLOLI...,NaN
2,Senior Game Analyst,"Analytical Skills, Data Analysis, Economics, M...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3717037977/...,"Next Games, A Netflix Game Studio",https://www.linkedin.com/company/next-games/life,"Next Games, A Netflix Game Studio · Helsinki, ...",Reposted 6 days ago,0,About the job\nNext Games is a Netflix Game St...,NaN
3,Data Scientist,"Data Analysis, Data Science, Machine Learning,...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3735986015/...,MedEngine,https://www.linkedin.com/company/medengine/life,"MedEngine · Helsinki, Uusimaa, Finland 1 week...",1 week ago,0,About the job\nMedEngine is a digitally minded...,NaN
4,Data Science - Machine Learning Engineer,"Artificial Intelligence (AI), Computer Science...",Remote,Full-time,https://www.linkedin.com/jobs/view/3629670334/...,Wolt,https://www.linkedin.com/company/wolt-oy/life,"Wolt · Helsinki, Uusimaa, Finland Reposted 2 ...",Reposted 2 weeks ago,0,About the job\nJob Description\n\nTeam purpose...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
307,Remote Work (Finnish Speakers) - Internet Ads ...,English and Finnish,Remote,NaN,https://www.linkedin.com/jobs/view/3728766620/...,TELUS International AI Data Solutions,https://www.linkedin.com/company/telusinternat...,TELUS International AI Data Solutions · Finlan...,3 weeks ago,0,About the job\nOur Company \n\nTELUS Internati...,NaN
308,Work From Home - Finnish Speakers (Internet Ad...,English and Finnish,Remote,NaN,https://www.linkedin.com/jobs/view/3731394329/...,TELUS International AI Data Solutions,https://www.linkedin.com/company/telusinternat...,TELUS International AI Data Solutions · Finlan...,2 weeks ago,0,About the job\nOur Company \n\nTELUS Internati...,NaN
309,"Senior Engineering Manager (Bangkok based, rel...","Software DevelopmentC#, Engineering Management...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3616699365/...,Agoda,https://www.linkedin.com/company/agoda/life,"Agoda · Helsinki, Uusimaa, Finland Reposted 3...",Reposted 3 days ago,0,About the job\nAbout Agoda\n\nAgoda is an onli...,NaN
310,"Commercial Director Nordics, Transport Reagents","SwedishBusiness Planning, Commodities, Indirec...",On-site,Full-time,https://www.linkedin.com/jobs/view/3736132257/...,Yara Suomi,https://www.linkedin.com/company/yarasuomi/life,"Yara Suomi · Espoo, Uusimaa, Finland 1 week a...",1 week ago,0,About the job\nAbout Yara Industrial Solutions...,NaN


### Is there duplicate postings?

In [3]:
len(_data0.linkedin_url.unique())

312

### How are the descriptive statistics?

In [8]:
_data0.describe(include="all")

,job_title,required_skills,job_type_1,job_type_2,linkedin_url,company,company_linkedin_url,location,posted_date,applicant_count,job_description,benefits
count,312,311,312,269,312,312,312,312,312,312.0,312,12
unique,257,266,4,4,312,128,128,310,23,NaN,263,3
top,Remote Data Contributor – Image collection,Artificial Intelligence (AI) and Defining Requ...,Hybrid,Full-time,https://www.linkedin.com/jobs/view/3433869418/...,Tietoevry,https://www.linkedin.com/company/tietoevry/life,"Tietoevry · Jyväskylä, Central Finland, Finlan...",1 week ago,NaN,About the job\nPosition: Data Contributor \nP...,Benefits found in job post\nMedical insurance
freq,7,6,135,252,1,31,31,2,74,NaN,6,6
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


### Conclusions #1

- 

## 2. How many jobs require Finnish? English? Swedish? Or both Fin & Eng?

1. Detect job posting languages.

2. Should we 

In [1]:
! pip install fasttext-langdetect

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext-langdetect: filename=fasttext_langdetect-1.0.5-py3-none-any.whl size=7522 sha256=20517643b16b93f8951c8570f04fb82d93aa2e15091048028c63d9c233bb5c0b
  Stored in directory: /home/parallels/.cache/pip/wheels/e6/5b/5d/47e2fd5c2ff1028722739ce35f365e8f6eeb89ec97aa63e621
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199801 sha256=38032eb577f2af9b2f41a4ac6b1ba5ba30c0d8773ddf118c5c6ffce6a48aadd0
  Stored in directory: /home/parallels/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext-langdetect fasttext


In [ ]:
from ftlangdetect import detect

_data0.job_description.map(detect)

## 3. Job Classification - Data scientist, engineer or analyst (or None of those)?

- Naive approach - Match strings from the job title.
- More advanced approach - Cluster job descriptions.
- Some jobs are not related to any of the above. Drop those.

### 3.1 Naive approach

In [9]:
len(_data0.loc[_data0.job_title.str.lower().str.contains("engineer"),:])

62

In [10]:
len(_data0.loc[_data0.job_title.str.lower().str.contains("scientist"),:])

17

In [11]:
len(_data0.loc[_data0.job_title.str.lower().str.contains("analyst"),:])

29

In [18]:
_none_of_three = ~(_data0.job_title.str.lower().str.contains("engineer") |
                   _data0.job_title.str.lower().str.contains("scientist") |
                    _data0.job_title.str.lower().str.contains("analyst"))
_data0.loc[_none_of_three,:]

,job_title,required_skills,job_type_1,job_type_2,linkedin_url,company,company_linkedin_url,location,posted_date,applicant_count,job_description,benefits
10,"Senior Legal and Privacy Counsel, Veho Group","Data Protection Law, Legal Document Preparatio...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3732885339/...,Veho Oy Ab,https://www.linkedin.com/company/veho-group-oy...,"Veho Oy Ab · Vantaa, Uusimaa, Finland 2 weeks...",2 weeks ago,0,About the job\nWe are looking for a Senior Leg...,NaN
12,Data Platform and AI Trainees,"Analytical Skills, Data Engineering, and Data ...",On-site,Full-time,https://www.linkedin.com/jobs/view/3741073031/...,OP Financial Group,https://www.linkedin.com/company/op-financial-...,"OP Financial Group · Helsinki, Uusimaa, Finlan...",6 days ago,0,About the job\nThe application period for OP F...,NaN
18,ETL Specialist,"Data Warehousing, English, Extract, Transform,...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3743271896/...,Gazelle Global,https://www.linkedin.com/company/gazelle-globa...,"Gazelle Global · Helsinki, Uusimaa, Finland 2...",2 days ago,0,About the job\nETL Specialist\n\n A great oppo...,NaN
25,Cloud & Enterprise Architect,"Access Management, Azure Functions, Azure Kube...",Hybrid,Full-time,https://www.linkedin.com/jobs/view/3735898635/...,Samlink – A Kyndryl Company,https://www.linkedin.com/company/oy-samlink-ab...,"Samlink – A Kyndryl Company · Espoo, Uusimaa, ...",1 week ago,0,About the job\nAre you a seasoned \n\nCloud & ...,NaN
26,Digital Analytics Specialist - Data Activation,"Analytics, Data Analytics, Data Engineering, D...",On-site,Full-time,https://www.linkedin.com/jobs/view/3696872830/...,Nordea,https://www.linkedin.com/company/nordea/life,"Nordea · Helsinki, Uusimaa, Finland Reposted ...",Reposted 3 days ago,0,About the job\nJob ID: 19131 \nWould you like ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Manager/ Senior Manager, Social MediaTeam (Ban...","Digital Marketing ChannelsDigital Marketing, E...",Full-time,NaN,https://www.linkedin.com/jobs/view/3702377220/...,Agoda,https://www.linkedin.com/company/agoda/life,"Agoda · Helsinki, Uusimaa, Finland Reposted 1...",Reposted 1 week ago,0,About the job\nAbout Agoda\n\nAgoda is an onli...,NaN
307,Remote Work (Finnish Speakers) - Internet Ads ...,English and Finnish,Remote,NaN,https://www.linkedin.com/jobs/view/3728766620/...,TELUS International AI Data Solutions,https://www.linkedin.com/company/telusinternat...,TELUS International AI Data Solutions · Finlan...,3 weeks ago,0,About the job\nOur Company \n\nTELUS Internati...,NaN
308,Work From Home - Finnish Speakers (Internet Ad...,English and Finnish,Remote,NaN,https://www.linkedin.com/jobs/view/3731394329/...,TELUS International AI Data Solutions,https://www.linkedin.com/company/telusinternat...,TELUS International AI Data Solutions · Finlan...,2 weeks ago,0,About the job\nOur Company \n\nTELUS Internati...,NaN
310,"Commercial Director Nordics, Transport Reagents","SwedishBusiness Planning, Commodities, Indirec...",On-site,Full-time,https://www.linkedin.com/jobs/view/3736132257/...,Yara Suomi,https://www.linkedin.com/company/yarasuomi/life,"Yara Suomi · Espoo, Uusimaa, Finland 1 week a...",1 week ago,0,About the job\nAbout Yara Industrial Solutions...,NaN


## 4. Explore Job type: remote, on-site or hybrid? 

Can we always tell? Is there any missing value?

## 5. Explore Job type: full-time, contract, part-time or internship?

Can we always tell? Is there any missing value?